# Intermediate Python programming - modules, data containers and regular expressions

Today we will spend a small time on external code (modules/packages), a medium amount of time on data containers, and most of the session on practising regular expressions.

# Modules
In Python, third party or external code is called a 'module' or 'package'. These refer to  organised code written by others to perform particular tasks. The Python language as standard comes with many additional modules that are useful for a range of commonly-performed tasks, e.g. manipulating regular expressions, interacting with the operating system and command-line, etc. There is a _huge_ ecosystem of modules available, many of which are available through the PyPI repository: https://pypi.org/.

We will use many of these modules in todays and future workshops. Code sharing is a standard practise in research, because in science we have to be transparent. If other researchers can't replicate our analyses then it reduces the quality of our science and the impact of our work. In contrast, private companies often have to strike a balance between intellectual property and transparency.

Loading a module into your session or script is very simple:

In [1]:
import os

This makes the functions in the `os` module available to you. We reference them with 'package.function', for example:

In [3]:
os.listdir()

['Code_example.png',
 '.DS_Store',
 'Workshop 1 - An introduction to Python programming.ipynb',
 'ExerciseData',
 'Code_markdown_menu.png',
 'Markdown_example.png',
 'Workshop 2.ipynb',
 'syntax_highlight.png',
 'kernel_restart.png',
 '.ipynb_checkpoints',
 'Workshop 1 - An introduction to Python programming.html',
 '2023-08-24_getting_started.docx']

The output here is the current working directory (this is the default).

Some modules have sub-modules, and we use further full-stops to access the functions in these sub-modules.

In [8]:
os.path.exists("/")

True

Here the sub-module of `os` is `path`, which provides functions for checking file paths on your operating system.

__Exercise break__
Spend 5 minutes looking up some modules in PyPI under [bioinformatics](https://pypi.org/search/?q=&o=&c=Topic+%3A%3A+Scientific%2FEngineering+%3A%3A+Bio-Informatics). You'll find they have lots of information about the modules, including a description of the functionality, documentation and who the creator and maintainers. Some will also have a link to the the application programming interface (API) - this is how many programmers find out how to use specific module functions, and is what is generated when you use the `help()` function.

# Data containers - numpy and pandas
We've already seen the basic (in-built) data container types: list, dicts and tuples. However, these are limited by several factors: 1) none of them represent an intuitive table-like data structure, 2) nested lists of lists or dicts of lists can become very unweildy very quickly, 3) none are optimised for memory and access performance.

This is where 2 modules come to our aid: `pandas` gives us `data frames` (i.e. tables) and `numpy` gives us `arrays` (i.e. more data tables). At first you might wonder why we need both. The answer is that pandas is _built on top of numpy_. That means that numpy came first and goes beyond simple data tables (arrays are more like matrices from linear algebra) and can extend to multiple dimensions but are limited by only being allowed to contain a single type of data, e.g. all floats or all strings. In contrast, data frames are more flexible and contain mixtures of data types in their columns, much more like an excel spreadsheet, and therefore more intuitive.

## Indexing and slicing
A quick aside on how to index/slice in Python, as this applies to both arrays and data frames. Indexing refers to selecting a specific element or set of elements from a data container. Lists, tuples, strings and arrays can be indexed or sliced, where as sets and numbers cannot.

To index we select a single element using square brackets '[ ]' and a number:

In [13]:
myList = ["Mike", "Ellie", "Kim", "Luna"]
myList[0]
# note that Python is a 0-based language

'Mike'

To take a slice we use the same square brackets, but we provide the index for the first element and number _after_ the last element. This is because Python uses an right-hand closed system, so the number at the end of the slice _is not included_.

In [15]:
myList[1:3]
# note that index 3 is NOT included and the '1' index is the 2nd element because Python starts counting at 0!

['Ellie', 'Kim']

We can also skip elements by taking a slice with a step size (the default is 1, and therefore is _implicit_).

In [17]:
myList[::2] # the first 2 indices are empty, which just defaults to the first and last

['Mike', 'Kim']

You can also include negative numbers as indices, for instance to select the last element:

In [18]:
myList[-1]

'Luna'

You can also reverse a list using slicing:

In [48]:
myList[::-1]

['Luna', 'Kim', 'Ellie', 'Mike']

Note that indices have to be whole numbers, you can't use a fraction (float):

In [49]:
myList[0.42]

TypeError: list indices must be integers or slices, not float

__Exercise break__: Create a list and try out some of the indexing and slicing. What happens if you use a list of lists?

## Numpy arrays
Numpy arrays must contain a single data type, e.g. float, integer or string. They can be used as vectors 1 x N dimensions, matrices N x M dimensions or arrays N x M x P dimensions.

Example usage. Imagine we have 5 DNA sequences that we want to align using BLAST, and then record whether the nucleotides match at each position or not. We could have 5 lists that record which positions match against which other sequences at each nucleotide position. We would need a pair of lists for each pair of sequences - this could get very messy if we had very long DNA sequences or indeed more than 5 to compare. Instead, we could record for each pair of sequences which nucleotides match using an array.

In [39]:
import numpy as np
dna_seqs = ['ATGGGACTGCCCTTGATGCCTCACGTATCCAAAAGCGTATCATGGAGCACTCGGAAACGCACTATAGCAATACCAGACCGTGATATACAACCAGATCACTCTACGATTAAACGACTGCGCCGCCGTATTGCGCTGCTTATCCGAGAAGCAGTCCCTCGTGCGACGCAGCACCAAGAACAAGTCTCAGAGTACCTTTGGACGTATGACTTCAGGATCGAGTCGGCGGCTAGGTTCTATGGGGAGCTCCGTGATAGGTCATCTAGACAGGGTGTGTATTATTTGAGAGTAAAACGAAATCCTCCTCGCATGTACGGGTCAATGGAATTCCCGGTCAGA',
            'ATGGACACCTGGACATACTCCCAAGAGGCTGGAGCAGTGCCCCATTCCTCGGTGCCTCAAGTTCTGGGAATGTGCAATCCTCCGCGAGACATTCAAGGTTTCAGACCACGCGATCGGGTTAAGATACTATCTCAAGTACTTCTTTCGTGTAAGGCACCACTAGAATTGCTTACTTATCCAGGCGGACAAACTGCAATCCTATCTTCATGGGGGGTAGATTTACCATTTTGTAAGAATCTAGACGCACCACGTGGGCGTAGCGACAGGGAGACACAATGGAGCTGCATGATATGGCGGGATCCAAGATCCATGGCGAGCGCGTCATACTTTAGGTGC',
            'ATGGTGCGCACGATTGAGTTGGAATTAGTCCCCAGACTGGCGACCATTACATACACCCTCCACCTTCCACGTCGGTTCAGCACGGGGTGGTGTTTCTCTCCAACACGAGGCCCGGGCGAATTAGTATTGATAATGCCTGGGCAGACAGCACTGGCCCGGCAGGAGATACTGGGCGATGGAAGACGTGGGTTAAGGCACGTCGGTTCTCTAGGTTTTCTATACTGGGCACGGAAGTTAGGGGCTAAAGAGCAACATACCTCTCCGCTGTCGGTCGATAGTGAGTCTTGCATAGCGATTGCTAAGAGCCTAGTTAAGCATACTCGTAACCGCGCCGCG',
            'ATGCCTCCCGGTATCGCTGAGGCCCTAGCAAACGCACAGATCAACCAACTTGTGTCTAACGCCATGGGATCTGCCGCATTAGCTAACAATATTACGCCTATAACGCACGGCTTACCTTTCAAATGTGCTGACCAAGTGGCCTCGATACAATTAGGGACTACAGGGGATAGATGTGCAAGTTTGTTTCTATCAGCGCCGGTGATTCTACGACCGTGTAATCAGAGGGCAGCGAAGATGGATAAGGGGTTGACCAACATGCTCAGTAATGATCCAGGAGCCGGTTATGCCATGCGAATATGCATGTGCACAGTAGCCGGGAGAGCTTGTATTCGTATA',
            'ATGACCCCACGCGGACCGTCAGGAGAAATCCTCTCACGGGATCCCGTCACTGTAACGCTCAGGGTCTTACGGCAACTCGAATGCTGTCGAAAATGTGGCAGTCAATCTCTATACCGGGGTCCGAGCTTTTTACATGTAATTTTGCAAGAATTCTTATATAGCACGATGAGACACGTACCCGATCACTTCGCCCAGCATACACTGATTAATACCGTCTATGTGATGTATTTACTTAAGACTCTCGCAACACCCGTGGCTTGGCGAAGTCTATCTGCACTAGTCGGGTCAGCCGTGCGGCGCGCACACACATCAAAGTGGCCGCAGTTGAGCGCAGTG']

len(dna_seqs[0])

336

We create a 5 x 5 x 336 array.

In [40]:
dna_array = np.ndarray((5, 5, 336))
dna_array

array([[[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000]],

       [[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         0.00000000e+000, 0.00000000e+000, 0.000000

This array has random numbers in it because it is assigned to specific piece of memory, so takes on the values in that memory block. It is better practise, and therefore less accident prone, to create an all-zeros array.

In [41]:
dna_array = np.zeros((5, 5, 336))
dna_array

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [42]:
dna_array.shape

(5, 5, 336)

We can now use this to store which nucleotides match for each pair of sequences.

In [43]:
for idx1 in range(len(dna_seqs)):
    # loop 1 gets each seq
    # loop 2 gets the other seq to compare to.
    seq1 = dna_seqs[idx1]
    for idx2 in range(len(dna_seqs)):
        seq2 = dna_seqs[idx2]
        # assume the sequences are the same length
        for nt in range(len(seq1)):
            if seq1[nt] == seq2[nt]:
                dna_array[idx1, idx2, nt] = 1
            else:
                dna_array[idx1, idx2, nt] = 0 # we don't actually need this because the default value is already 0

In [46]:
dna_array[1, 1, :] # sequence 1 matches perfectly to itself

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [47]:
dna_array[1, 4, :] 

array([1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 1.

__Exercise break__
Can you find the 2 sequences that have the closest match to each other? How many sequences match at the first nucleotide, or the 100th?

## Data frames
Data frames are provided by the `pandas` Python module: https://pandas.pydata.org/

These are N x M dimension tables which allow you to index the rows or columns, and store mixed data types in the columns. Note that each column must still contain a single type of data, i.e. integer, float or string.

Data frames can be created from lists, dicts and/or numpy arrays. There are also functions to read data from a files directly into a data frame.

### Data frame from list, dicts and arrays

In [55]:
import pandas as pd

In [75]:
phone_book = {"Names": ["Mike", "Daniel", "Ellie", "Heather"],
             "Office": ["6.17", "5.23", "3.20", "3.18"],
             "Department": ["Immunology", "Neuroscience", "Developmental Biology", "Immunology"]}
pb_df = pd.DataFrame(phone_book)

![Anatomy of a data frame](df_anatomy.png)

Each row has an index - here it is a number (default), but this can also be a string which uniquely identifies each row, for example a sample identifier. Each column then contains data in the same type (string, float, string) corresponding to the data in our data frame.

We can index the data frame by either the columns or the rows to return a `Series`. To index the rows we use the `.loc` and `.iloc` functions.

In [76]:
type(pb_df.iloc[0, :])

pandas.core.series.Series

In [77]:
pb_df.iloc[0, :]

Names               Mike
Office              6.17
Department    Immunology
Name: 0, dtype: object

The columns can be accessed/sliced using the names or the column indices.

In [78]:
pb_df.iloc[:, 0] # the first column

0       Mike
1     Daniel
2      Ellie
3    Heather
Name: Names, dtype: object

In [79]:
pb_df["Names"]

0       Mike
1     Daniel
2      Ellie
3    Heather
Name: Names, dtype: object

In [80]:
pb_df.iloc[:, 0] == pb_df["Names"] # these are equivalent.

0    True
1    True
2    True
3    True
Name: Names, dtype: bool

We can set the row index (if it is only the default) using the `.set_index()` function. We have to tell the function which column to create the index from, whether to then keep that column in the data frame (the default is to drop it), and if we want to return a new dataframe or perform the operation _in place_ (the default is to return a new data frame).

In [83]:
pb_df.set_index("Names", inplace=True, drop=False)
pb_df

,Names,Office,Department
Names,,,
Mike,Mike,6.17,Immunology
Daniel,Daniel,5.23,Neuroscience
Ellie,Ellie,3.20,Developmental Biology
Heather,Heather,3.18,Immunology


We can now access the rows of the data frame using the row indices (because they are unique).

In [84]:
pb_df.loc["Mike", :]

Names               Mike
Office              6.17
Department    Immunology
Name: Mike, dtype: object

In [85]:
pb_df.loc[["Mike", "Heather"], :] # selecting multiple rows returns a data frame.

,Names,Office,Department
Names,,,
Mike,Mike,6.17,Immunology
Heather,Heather,3.18,Immunology


### Data frame from a list of lists
We could make a data frame from a list of lists. Each sub-list must contain a single data type. Each list needs to be the same length.

In [87]:
phone_list = [["Mike", "Daniel", "Ellie", "Heather"],
              ["6.17", "5.23", "3.20", "3.18"],
              ["Immunology", "Neuroscience", "Developmental Biology", "Immunology"]]
list_df = pd.DataFrame(phone_list)
list_df

,0,1,2,3
0,Mike,Daniel,Ellie,Heather
1,6.17,5.23,3.20,3.18
2,Immunology,Neuroscience,Developmental Biology,Immunology


See how by default the data frame puts each list into a row. Why do you think this is the default behaviour?

### Data frame from a file
We can also read in data from a text file that is in tabular format, e.g. comma or tab-separated. It is better to store data this way than in Excel spreadsheets to minimise the chances of data corruption.

In [91]:
phone_from_file = pd.read_csv("phone_book.csv", sep=",")
phone_from_file

,Names,Office,Department
0,Mike,6.17,Immunology
1,Daniel,5.23,Neuroscience
2,Ellie,3.20,Developmental Biology
3,Heather,3.18,Immunology


__Exercise break__
Try making your own data frames from dicts, lists or reading in from a files. For example, you could use the DNA sequences above and summarise some information about, e.g. how many A's, T's, C's and G's they each contain. Can you figure out how to store this information in a data frame?

# Regular expressions
These are an incredibly powerful way to perform pattern matching, but take some practise to get used to. Recall from the lecture the table of the regex syntax - try to memorise the major ones (top 11) - you can always keep a cheat sheet of these to hand - or google them, whichever you prefer.

In Python, we use the `re` module for regular expressions: https://docs.python.org/3/library/re.html

In [93]:
import re

We start by constructing a regular expression, then we apply it to some text. In our case we want to identify TATA boxes in a set of bacterial promoter sequences. Recall from the lecture that a TATA box is 5'-TATAT/AAT/A-3', where the forward slash means one nucleotide or another, so either T or A in both cases.

In [95]:
# we make our regex then convert it into a regex object using re.compile
tata_re = re.compile("TATA[A|T]A[A|T]")
tata_re

re.compile(r'TATA[A|T]A[A|T]', re.UNICODE)

In [98]:
bact_seq = "CACTTCAGGTTAAGCGGCGCGGCCTGCGCATTTTGGATATATATAATCCT"
tata_match = tata_re.search(bact_seq)
tata_match

<re.Match object; span=(37, 44), match='TATATAT'>

The result here is an `re.Match` object (more on objects later, if there's time). It contains some specific information already, namely the position in the string of the regex match (37-44) and the exact sequence. We can extract this information from `tata_match` and use it to index the relevant sequence.

In [102]:
tata_idx = tata_match.span()
bact_seq[tata_idx[0]:tata_idx[1]] # tata_idx[0] is the first position, tata_idx[1] is the last position(+1)

'TATATAT'

In [109]:
tata_match.group(0) # this syntax is because we can have multiple groups in a regex

'TATATAT'

__Exercise break__
Using the DNA sequences from above in `dna_seqs`, can you find the start and stop codons? Can you find all of the instances of the lysine codons? How many lysine codons are there in each of the 5 sequences?

## Final exercise

You have been provided with a file containing 10 cDNA sequences. For each one transcribe them into RNA then translate into peptide sequences - note that you will need to find the longest reading frame first. Think about what you know about where translation is started. I want you to use regular expressions to find all of the serine and threonine codons and count them. Note that some sequences may not contain _any_ serine or threonine amino acids. For each sequence store the data in a data frame, along with the GC percentage for each original DNA sequence.